In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import scipy.sparse as sp

# Load the dataset
dataset = pd.read_csv("features_dataset.csv")

# Separate features and labels
X = dataset.drop(columns=['label'])
y = dataset['label']

# Split the dataset into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training set into a smaller subset
X_train, _, y_train, _ = train_test_split(X_train_full, y_train_full, train_size=0.25, random_state=42)

# Vectorize the '1_grams' column
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_1grams = vectorizer.fit_transform(X_train['1_grams'])
X_test_1grams = vectorizer.transform(X_test['1_grams'])

# Drop the '1_grams' column from the original dataset
X_train = X_train.drop(columns=['1_grams'])
X_test = X_test.drop(columns=['1_grams'])

# Concatenate the vectorized '1_grams' column with the remaining features
X_train = sp.hstack((X_train.values, X_train_1grams), format='csr')
X_test = sp.hstack((X_test.values, X_test_1grams), format='csr')

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest classifier
random_forest = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Best Score: 0.9909616775126536
Accuracy: 0.9931333574268161
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1353
           1       0.99      0.99      0.99      1414

    accuracy                           0.99      2767
   macro avg       0.99      0.99      0.99      2767
weighted avg       0.99      0.99      0.99      2767



In [3]:
# Calculate training accuracy
train_accuracy = best_model.score(X_train, y_train)

# Calculate testing accuracy
test_accuracy = best_model.score(X_test, y_test)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)


Training Accuracy: 1.0
Testing Accuracy: 0.9931333574268161


In [11]:
import numpy as np
# Assuming you have already trained your Random Forest model called best_model

# Get feature importances
importances = best_model.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


Feature ranking:
1. feature 3850 (0.132251)
2. feature 3956 (0.044309)
3. feature 2270 (0.024941)
4. feature 2 (0.013213)
5. feature 2683 (0.008918)
6. feature 2562 (0.007936)
7. feature 1738 (0.007521)
8. feature 4880 (0.007507)
